In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [187]:
from water_pollution.api import ann_utils as util

from fastapi import FastAPI
import requests

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np

from dotenv import load_dotenv
import os

In [3]:
load_dotenv() # Cherche le fichier .env dans le projet et charge les variables d'environnement
key = os.getenv('WEATHERKEY')

In [65]:
## Argument de la fonction
station_id = 6059500

In [ ]:
## Stations DF, used below
stationsdf = util.get_stations_df()

In [66]:
## Coordonnées pour requete API

try :
    coords = stationsdf.loc[station_id,'coord']
    lat,lon = coords[0],coords[1]
    coord_str = str(lat) + ',' + str(lon)
    
except :
    # return {'error':'ID de la station inxonnu'}
    pass


In [172]:
#### API REQUESTS

# General params

key = os.getenv('WEATHERKEY')
url_hist = 'http://api.weatherapi.com/v1/history.json'
url_prev = 'http://api.weatherapi.com/v1/forecast.json'

# Built list of weather information
weather_list = []


# API FORECAST REQUEST

params = {
    'key':key,
    'q':coord_str,
    'days':15,
}

response = requests.get(url_prev,params=params)

if not response :# if response is not 200
    pass # traiter les exceptions...

jr = response.json()

forecast = jr.get('forecast').get('forecastday')

for f in forecast :
    weather_list.append((
        f.get('date'),
        f.get('day').get('avgtemp_c'),
        f.get('day').get('totalprecip_mm')
    ))

# API HISTORY REQUESTS

# Total date range
now = datetime.now()
month_first = now.replace(day=1)
month_m4_first = month_first - relativedelta(months=5)

date_range = pd.date_range(start=month_m4_first,end=now,freq='D')

# Date ranges by month
date_ranges = []
for m in set(date_range.month):
    date_ranges.append( date_range[date_range.month == m] )
    
# Ranges
string_ranges = []

for dr in date_ranges :
    dt = dr[0].strftime("%Y-%m-%d")
    end_dt = dr[-1].strftime("%Y-%m-%d")
    string_ranges.append((dt,end_dt))
    
for string_range in string_ranges :
    
    params = {
        'key':key,
        'q':q,
        'dt':string_range[0],
        'end_dt':string_range[1],
        }
    
    response = requests.get(url_hist,params=params)

    if not response : # if response is not 200
        pass # traiter les exceptions...

    jr = response.json()

    forecast = jr.get('forecast').get('forecastday')

    for f in forecast :
        
        weather_list.append((
            f.get('date'),
            f.get('day').get('avgtemp_c'),
            f.get('day').get('totalprecip_mm')
        ))

In [173]:
## BUILDING FINAL DF

weatherdf = pd.DataFrame(weather_list)
weatherdf.columns = ['date','temp','precipitation']
weatherdf['date'] = pd.to_datetime(weatherdf['date']) 
weatherdf.drop_duplicates('date',inplace=True)
weatherdf.set_index('date',inplace=True)

In [189]:
# Does the windows and shifts

precdf = weatherdf[['precipitation']].copy()
precdf = precdf.rolling(window=35).mean().dropna() # Window of 40
precdf.index = precdf.index + timedelta(15) # delta of 15

tempdf = weatherdf[['temp']].copy()
tempdf = tempdf.rolling(window=40).mean().dropna() # Window of 40
tempdf.index = tempdf.index + timedelta(15) # delta of 15

weatherdf = pd.merge(precdf,tempdf,left_index=True,right_index=True)

# Adds the Nitrate Mean of the station
weatherdf['mean_station'] = stationsdf.loc[station_id,'mean_nitrate']

# Adds the day of year sin and cos
weatherdf['sin_doy'] = np.sin( (weatherdf.index.day_of_year-1) * 2 * np.pi / 365 )
weatherdf['cos_doy'] = np.cos( (weatherdf.index.day_of_year-1) * 2 * np.pi / 365 )

In [194]:
weatherdf

,precipitation,temp,mean_station,sin_doy,cos_doy
date,,,,,
2021-09-03,3.873878,20.880688,8.835948,-0.880012,-0.474951
2021-09-04,3.925143,21.033875,8.835948,-0.888057,-0.459733
2021-09-05,3.977878,21.172187,8.835948,-0.895839,-0.444378
2021-09-06,4.054286,21.293187,8.835948,-0.903356,-0.428892
2021-09-07,4.114367,21.401438,8.835948,-0.910605,-0.413279
...,...,...,...,...,...
2021-12-25,0.936000,13.192438,8.835948,-0.120208,0.992749
2021-12-26,0.908898,13.028062,8.835948,-0.103102,0.994671
2021-12-27,0.885633,12.863938,8.835948,-0.085965,0.996298
